In [ ]:
!pip install transformers datasets evaluate accelerate

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch

In [ ]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
imdb = load_dataset("imdb")

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

tokenized_imdb = imdb.map(preprocess_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

accuracy = evaluate.load("accuracy")

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.231300,0.210296,0.918360
2,0.154000,0.245181,0.929160


Epoch,Training Loss,Validation Loss,Accuracy
1,0.231300,0.210296,0.918360
2,0.154000,0.245181,0.929160
3,0.092100,0.284398,0.931640


TrainOutput(global_step=4689, training_loss=0.1697925076247736, metrics={'train_runtime': 4764.0807, 'train_samples_per_second': 15.743, 'train_steps_per_second': 0.984, 'total_flos': 9834539051060448.0, 'train_loss': 0.1697925076247736, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.284397691488266,
 'eval_accuracy': 0.93164,
 'eval_runtime': 404.3675,
 'eval_samples_per_second': 61.825,
 'eval_steps_per_second': 3.865,
 'epoch': 3.0}

In [ ]:
text = "Honestly, don't know how to feel about it"

tokenizer = AutoTokenizer.from_pretrained("results/checkpoint-4689")
inputs = tokenizer(text, return_tensors="pt")

model = AutoModelForSequenceClassification.from_pretrained("results/checkpoint-4689")
with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

'NEGATIVE'

In [ ]:
model.save_pretrained("./sentiment-model")
tokenizer.save_pretrained("./sentiment-tokenizer")

('./sentiment-tokenizer/tokenizer_config.json',
 './sentiment-tokenizer/special_tokens_map.json',
 './sentiment-tokenizer/vocab.txt',
 './sentiment-tokenizer/added_tokens.json',
 './sentiment-tokenizer/tokenizer.json')

In [ ]:
!zip -r /content/sentiment_model.zip /content/sentiment-model/
!zip -r /content/sentiment_tokenizer.zip /content/sentiment-tokenizer/

updating: content/sentiment-model/ (stored 0%)
updating: content/sentiment-model/model.safetensors (deflated 8%)
updating: content/sentiment-model/config.json (deflated 45%)
  adding: content/sentiment-tokenizer/ (stored 0%)
  adding: content/sentiment-tokenizer/special_tokens_map.json (deflated 42%)
  adding: content/sentiment-tokenizer/tokenizer_config.json (deflated 76%)
  adding: content/sentiment-tokenizer/tokenizer.json (deflated 71%)
  adding: content/sentiment-tokenizer/vocab.txt (deflated 53%)
